In [34]:
import os
import re
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.lines import Line2D

In [35]:
fss_path = "/home/users/mendrika/Zambia-Intercomparison/comparison/fss/data"
out_dir = "/home/users/mendrika/Zambia-Intercomparison/comparison/fss/figs_zcast_netncc_avg3h_tab10_topright"
os.makedirs(out_dir, exist_ok=True)

lead_times = [1, 2, 4, 6]

In [36]:
files = [f for f in os.listdir(fss_path) if f.endswith(".csv")]
pattern = r"fss_hour_(\d{2})_t(\d)\.csv"

In [45]:
all_dfs = []
for fname in files:
    m = re.match(pattern, fname)
    if not m:
        continue
    hour = int(m.group(1))
    lt = int(m.group(2))
    df = pd.read_csv(os.path.join(fss_path, fname))
    df["hour"] = hour
    df["lead_time"] = lt

    # round scale to nearest 10 km
    if "scale_km" in df.columns:
        df["scale_km"] = (df["scale_km"] / 10).round() * 10

    all_dfs.append(df)

if not all_dfs:
    raise RuntimeError("No Fraction Skill Score CSV files found. Check path or pattern.")

fss_all = pd.concat(all_dfs, ignore_index=True)


fss_all["hour_bin"] = (fss_all["hour"] // 3) * 3
palette = plt.colormaps.get_cmap("tab10")

for lt in lead_times:
    df_lt = fss_all[fss_all["lead_time"] == lt]
    if df_lt.empty:
        print(f"No data for lead time t{lt}")
        continue

    # average within 3-hour bins
    df_mean = (
        df_lt.groupby(["hour_bin", "scale_km"], as_index=False)[["zcast", "netncc"]]
        .mean()
        .sort_values("hour_bin")
    )

    scales = sorted(df_mean["scale_km"].unique())
    n_scales = len(scales)
    colours = {s: palette(i / max(1, n_scales - 1)) for i, s in enumerate(scales)}

    plt.figure(figsize=(10, 6))
    for s in scales:
        df_s = df_mean[df_mean["scale_km"] == s]
        colour = colours[s]

        # NetNCC (solid, circle)
        plt.plot(df_s["hour_bin"], df_s["netncc"],
                 color=colour, linestyle="-", marker="o",
                 markersize=6, linewidth=2.2)

        # ZCAST (dashed, triangle)
        plt.plot(df_s["hour_bin"], df_s["zcast"],
                 color=colour, linestyle="--", marker="^",
                 markersize=5, linewidth=1.8)

    # reference line
    plt.axhline(y=0.5, color="grey", linestyle="--", linewidth=1, alpha=0.8)

    # axes and grid
    plt.xlabel("Hour of day (UTC, 3-hour bins)")
    plt.ylabel("Fraction Skill Score")
    plt.xticks(range(0, 24, 3))
    plt.ylim(0, 1)
    plt.grid(True, alpha=0.3, linewidth=0.8)

    # legends: colours = scales (top right), styles = models (top left)
    scale_handles = [Line2D([], [], color=colours[s], marker="s", linestyle="",
                            markersize=6, label=f"{int(s)} km") for s in scales]
    model_handles = [
        Line2D([], [], color="black", linestyle="-", marker="o", label="NetNCC (solid, ●)"),
        Line2D([], [], color="black", linestyle="--", marker="^", label="ZCAST (dashed, ▲)")
    ]
    first_leg = plt.legend(handles=scale_handles, title="Scale (km)", ncol=2, loc="upper right", fontsize=9)
    plt.gca().add_artist(first_leg)
    plt.legend(handles=model_handles, title="Model", loc="upper left", fontsize=9)

    plt.tight_layout()
    out_file = os.path.join(out_dir, f"fss_hour_netncc_zcast_t{lt}_avg3h_tab10.png")
    plt.savefig(out_file, dpi=300)
    plt.close()
    print(f"Saved: {out_file}")

Saved: /home/users/mendrika/Zambia-Intercomparison/comparison/fss/figs_zcast_netncc_nflics_avg3h_tab10_side/fss_hour_netncc_zcast_t1_avg3h_tab10.png
Saved: /home/users/mendrika/Zambia-Intercomparison/comparison/fss/figs_zcast_netncc_nflics_avg3h_tab10_side/fss_hour_netncc_zcast_t2_avg3h_tab10.png
Saved: /home/users/mendrika/Zambia-Intercomparison/comparison/fss/figs_zcast_netncc_nflics_avg3h_tab10_side/fss_hour_netncc_zcast_t4_avg3h_tab10.png
Saved: /home/users/mendrika/Zambia-Intercomparison/comparison/fss/figs_zcast_netncc_nflics_avg3h_tab10_side/fss_hour_netncc_zcast_t6_avg3h_tab10.png


In [44]:
import os
import re
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.lines import Line2D

# ---------------------------------------------------------------------
# Configuration
# ---------------------------------------------------------------------
fss_path = "/home/users/mendrika/Zambia-Intercomparison/comparison/fss/data"
out_dir = "/home/users/mendrika/Zambia-Intercomparison/comparison/fss/figs_zcast_netncc_nflics_avg3h_tab10_side"
os.makedirs(out_dir, exist_ok=True)

lead_times = [1, 2, 4, 6]
palette = plt.colormaps.get_cmap("tab10")

# ---------------------------------------------------------------------
# Load all FSS CSV files
# ---------------------------------------------------------------------
files = [f for f in os.listdir(fss_path) if f.endswith(".csv")]
pattern = r"fss_hour_(\d{2})_t(\d)\.csv"

all_dfs = []
for fname in files:
    m = re.match(pattern, fname)
    if not m:
        continue
    hour = int(m.group(1))
    lt = int(m.group(2))
    df = pd.read_csv(os.path.join(fss_path, fname))
    df["hour"] = hour
    df["lead_time"] = lt

    # round scale to nearest 10 km
    if "scale_km" in df.columns:
        df["scale_km"] = (df["scale_km"] / 10).round() * 10

    all_dfs.append(df)

if not all_dfs:
    raise RuntimeError("No Fraction Skill Score CSV files found. Check path or pattern.")

fss_all = pd.concat(all_dfs, ignore_index=True)

# ---------------------------------------------------------------------
# Analysis: average every 3h, compare left (ZCAST + NetNCC) vs right (NFLICS)
# ---------------------------------------------------------------------
fss_all["hour_bin"] = (fss_all["hour"] // 3) * 3

for lt in lead_times:
    df_lt = fss_all[fss_all["lead_time"] == lt]
    if df_lt.empty:
        print(f"No data for lead time t{lt}")
        continue

    # average within 3-hour bins
    df_mean = (
        df_lt.groupby(["hour_bin", "scale_km"], as_index=False)[["zcast", "netncc", "nflics"]]
        .mean()
        .sort_values("hour_bin")
    )

    scales = sorted(df_mean["scale_km"].unique())
    n_scales = len(scales)
    colours = {s: palette(i / max(1, n_scales - 1)) for i, s in enumerate(scales)}

    # Two subplots: left (ZCAST + NetNCC), right (NFLICS)
    fig, axes = plt.subplots(1, 2, figsize=(14, 6), sharey=True)

    # --- Left: ZCAST and NetNCC ---
    ax = axes[0]
    for s in scales:
        df_s = df_mean[df_mean["scale_km"] == s]
        colour = colours[s]

        # NetNCC: solid, circle
        ax.plot(df_s["hour_bin"], df_s["netncc"], color=colour, linestyle="-",
                marker="o", markersize=6, linewidth=2.2)
        # ZCAST: dashed, triangle
        ax.plot(df_s["hour_bin"], df_s["zcast"], color=colour, linestyle="--",
                marker="^", markersize=5, linewidth=1.8)

    ax.axhline(y=0.5, color="grey", linestyle="--", linewidth=1, alpha=0.8)
    ax.set_xlabel("Hour of day (UTC, 3-hour bins)")
    ax.set_ylabel("Fraction Skill Score")
    ax.set_xticks(range(0, 24, 3))
    ax.set_ylim(0, 1)
    ax.grid(True, alpha=0.3, linewidth=0.8)
    ax.set_title("ZCAST vs NetNCC")

    # --- Right: NFLICS ---
    ax2 = axes[1]
    for s in scales:
        df_s = df_mean[df_mean["scale_km"] == s]
        colour = colours[s]
        ax2.plot(df_s["hour_bin"], df_s["nflics"], color=colour, linestyle="-",
                 marker="s", markersize=5, linewidth=2)

    ax2.axhline(y=0.5, color="grey", linestyle="--", linewidth=1, alpha=0.8)
    ax2.set_xlabel("Hour of day (UTC, 3-hour bins)")
    ax2.set_xticks(range(0, 24, 3))
    ax2.grid(True, alpha=0.3, linewidth=0.8)
    ax2.set_title("NFLICS")

    # --- Legends ---
    scale_handles = [Line2D([], [], color=colours[s], marker="s", linestyle="", markersize=6, label=f"{int(s)} km")
                     for s in scales]
    model_handles_left = [
        Line2D([], [], color="black", linestyle="-", marker="o", label="NetNCC (solid, ●)"),
        Line2D([], [], color="black", linestyle="--", marker="^", label="ZCAST (dashed, ▲)")
    ]
    model_handles_right = [
        Line2D([], [], color="black", linestyle="-", marker="s", label="NFLICS (solid, ■)")
    ]

    # Top right: scale legend (shared)
    first_leg = fig.legend(handles=scale_handles, title="Scale (km)",
                           ncol=3, loc="upper center", fontsize=9)
    # Left and right model legends
    axes[0].legend(handles=model_handles_left, title="Model", loc="upper left", fontsize=9)
    axes[1].legend(handles=model_handles_right, title="Model", loc="upper left", fontsize=9)

    plt.tight_layout(rect=[0, 0, 1, 0.93])
    out_file = os.path.join(out_dir, f"fraction_skill_netncc_zcast_nflics_t{lt}_avg3h_tab10.png")
    plt.savefig(out_file, dpi=300)
    plt.close()
    print(f"Saved: {out_file}")

Saved: /home/users/mendrika/Zambia-Intercomparison/comparison/fss/figs_zcast_netncc_nflics_avg3h_tab10_side/fraction_skill_netncc_zcast_nflics_t1_avg3h_tab10.png
Saved: /home/users/mendrika/Zambia-Intercomparison/comparison/fss/figs_zcast_netncc_nflics_avg3h_tab10_side/fraction_skill_netncc_zcast_nflics_t2_avg3h_tab10.png
Saved: /home/users/mendrika/Zambia-Intercomparison/comparison/fss/figs_zcast_netncc_nflics_avg3h_tab10_side/fraction_skill_netncc_zcast_nflics_t4_avg3h_tab10.png
Saved: /home/users/mendrika/Zambia-Intercomparison/comparison/fss/figs_zcast_netncc_nflics_avg3h_tab10_side/fraction_skill_netncc_zcast_nflics_t6_avg3h_tab10.png


In [47]:
# ---------------------------------------------------------------------
# Difference in Fraction Skill Score relative to NFLICS
# Averaged every 3 hours
# Legend and y-limit adjusted by lead time
# ---------------------------------------------------------------------
out_dir = "/home/users/mendrika/Zambia-Intercomparison/comparison/fss/figs_diff_vs_nflics_avg3h_legendpos_v3"
os.makedirs(out_dir, exist_ok=True)

# 3-hour bins
fss_all["hour_bin"] = (fss_all["hour"] // 3) * 3
palette = plt.colormaps.get_cmap("tab10")

for lt in [1, 2, 4, 6]:
    df_lt = fss_all[fss_all["lead_time"] == lt].copy()
    if df_lt.empty:
        print(f"No data for lead time t{lt}")
        continue

    # Compute difference from NFLICS
    df_lt["diff_zcast"] = df_lt["zcast"] - df_lt["nflics"]
    df_lt["diff_netncc"] = df_lt["netncc"] - df_lt["nflics"]

    # Average within 3-hour bins
    df_mean = (
        df_lt.groupby(["hour_bin", "scale_km"], as_index=False)[["diff_zcast", "diff_netncc"]]
        .mean()
        .sort_values("hour_bin")
    )

    scales = sorted(df_mean["scale_km"].unique())
    n_scales = len(scales)
    colours = {s: palette(i / max(1, n_scales - 1)) for i, s in enumerate(scales)}

    plt.figure(figsize=(10, 6))
    for s in scales:
        df_s = df_mean[df_mean["scale_km"] == s]
        colour = colours[s]

        # NetNCC (solid, circle)
        plt.plot(df_s["hour_bin"], df_s["diff_netncc"],
                 color=colour, linestyle="-", marker="o",
                 markersize=6, linewidth=2.2)

        # ZCAST (dashed, triangle)
        plt.plot(df_s["hour_bin"], df_s["diff_zcast"],
                 color=colour, linestyle="--", marker="^",
                 markersize=5, linewidth=1.8)

    # Reference line
    plt.axhline(y=0, color="grey", linestyle="--", linewidth=1, alpha=0.8)

    # Axes and grid
    plt.xlabel("Hour of day (UTC, 3-hour bins)")
    plt.ylabel("ΔFSS (Model − NFLICS)")
    plt.xticks(range(0, 24, 3))
    plt.grid(True, alpha=0.3, linewidth=0.8)

    # Adjust y-limits and legend positions by lead time
    if lt in [1, 2]:
        ylim = (-0.1, 0.7)
        scale_loc = "lower right"
        model_loc = "lower left"
    else:
        ylim = (-0.1, 0.5)
        scale_loc = "upper right"
        model_loc = "upper left"
    plt.ylim(ylim)

    # Legends
    from matplotlib.lines import Line2D
    scale_handles = [Line2D([], [], color=colours[s], marker="s", linestyle="",
                            markersize=6, label=f"{int(s)} km") for s in scales]
    model_handles = [
        Line2D([], [], color="black", linestyle="-", marker="o", label="NetNCC (solid, ●)"),
        Line2D([], [], color="black", linestyle="--", marker="^", label="ZCAST (dashed, ▲)")
    ]

    first_leg = plt.legend(handles=scale_handles, title="Scale (km)",
                           ncol=2, loc=scale_loc, fontsize=9)
    plt.gca().add_artist(first_leg)
    plt.legend(handles=model_handles, title="Model", loc=model_loc, fontsize=9)

    plt.tight_layout()
    out_file = os.path.join(out_dir, f"fraction_skill_diff_vs_nflics_t{lt}_avg3h.png")
    plt.savefig(out_file, dpi=300)
    plt.close()
    print(f"Saved: {out_file}")

Saved: /home/users/mendrika/Zambia-Intercomparison/comparison/fss/figs_diff_vs_nflics_avg3h_legendpos_v3/fraction_skill_diff_vs_nflics_t1_avg3h.png
Saved: /home/users/mendrika/Zambia-Intercomparison/comparison/fss/figs_diff_vs_nflics_avg3h_legendpos_v3/fraction_skill_diff_vs_nflics_t2_avg3h.png
Saved: /home/users/mendrika/Zambia-Intercomparison/comparison/fss/figs_diff_vs_nflics_avg3h_legendpos_v3/fraction_skill_diff_vs_nflics_t4_avg3h.png
Saved: /home/users/mendrika/Zambia-Intercomparison/comparison/fss/figs_diff_vs_nflics_avg3h_legendpos_v3/fraction_skill_diff_vs_nflics_t6_avg3h.png
